# Generate OD matrices for an underlying network

Using shortest path algorithmn to find shortest paths in Graph G (l_star)

In [9]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
#added to ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Read in underlying network

In [10]:
df = pd.read_csv("../networks/edge_list.csv")
df.head()

origin  destination      cost      lat_o      lon_o      lat_d      lon_d
0       0           43  0.829055  82.675559  25.254938  82.682786  82.682786
1       1            6  0.654921  82.819525  25.199418  82.825389  82.825389
2       1            9  0.541245  82.819525  25.199418  82.824264  82.824264
3       1           11  2.129843  82.819525  25.199418  82.800454  82.800454
4       1           12  0.288988  82.819525  25.199418  82.817047  82.817047

## Created directed graph

In [12]:
G = nx.from_pandas_edgelist(
    df, 'origin', "destination", "cost", create_using=nx.DiGraph())


## Determine shortest path

using cost (currenlty is length in km of an edge)

In [13]:
d = nx.floyd_warshall(G, weight="cost")

## Create new dataframe

In [14]:
final = pd.DataFrame(columns = ['origin', 'destination', 'l_star'])

## Append to l_star to df

In [16]:
rows = []
for i in d.keys():
    for j in d[i].keys():
        rows.append({'origin': i, 'destination': j, 'l_star': d[i][j]})
        #added line for determining edge frequency among paths 
        # if d[i][j] != 0:
        #     print(nx.dijkstra_path(G, i, j))


## Output to networks/l_star

In [20]:
final = pd.DataFrame(rows)
final = final[final['l_star']!=0].reset_index(drop = True)
#big number to represent inf.
final = final.replace(np.inf, 1000000) 
# final.to_csv('.networks/od/od.csv', encoding='utf-8', index=False)
final

origin  destination     l_star
0             0           43   0.829055
1             0            1  19.919975
2             0            6  19.554631
3             0            9  19.378730
4             0           11  22.040266
...         ...          ...        ...
1683501    1290         1284   1.159394
1683502    1290         1285   1.352113
1683503    1290         1286   1.602071
1683504    1290         1277   2.757112
1683505    1290         1274   0.519726

[1683506 rows x 3 columns]

## Add demand and priority to OD list

In [ ]:
# Add synthetic demand (positive integers) and priorities (random decimals between 0 and 1, excluding 0 and 1)
final['demand'] = np.random.randint(1, 100, size=len(final))
final['priority'] = np.random.uniform(0.001, 0.999, size=len(final))

In [ ]:
final.to_csv('../networks/l_star/india_od.csv', encoding='utf-8', index=False)